In [30]:
import numpy as np
import pandas as pd
import torch 
import torchvision
import transformers 
import datetime as dt

file not commented cleanly

## Filtering by Core Dates

import roberta scores

In [2]:
roberta_data = pd.read_csv('./data/prebuilt_rob_sentiment.csv')
roberta_data.head()

,Market Date,Publishing Time,Ticker,Sector,finvader_neg,finvader_neu,finvader_pos,finvader_tot,Source,Headline,...,Open,High,Low,Close,Volume,Dividends,Stock Splits,rob_sentiment,rob_score,fin_sentiment
0,2019-03-01,NaN,JNJ,Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,...,118.776213,119.979382,118.594433,119.754333,5641600,0.0,0.0,NaN,NaN,NaN
1,2019-03-01,NaN,JPM,Finance,NaN,NaN,NaN,NaN,NaN,NaN,...,89.960564,90.653889,89.190205,89.387077,13813300,0.0,0.0,NaN,NaN,NaN
2,2019-03-01,NaN,WFC,Finance,NaN,NaN,NaN,NaN,NaN,NaN,...,43.231147,43.756152,42.921307,43.059013,19051400,0.0,0.0,NaN,NaN,NaN
3,2019-03-01,NaN,V,Finance,NaN,NaN,NaN,NaN,NaN,NaN,...,144.164996,144.618332,143.123256,144.174637,7016800,0.0,0.0,NaN,NaN,NaN
4,2019-03-01,NaN,UNH,Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,...,225.135998,229.240138,225.135998,227.530075,6921300,0.0,0.0,NaN,NaN,NaN


In [3]:
roberta_data[roberta_data['Publishing Time'].notna()]
roberta_data.columns


Index(['Market Date', 'Publishing Time', 'Ticker', 'Sector', 'finvader_neg',
       'finvader_neu', 'finvader_pos', 'finvader_tot', 'Source', 'Headline',
       'Text', 'URL', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'rob_sentiment', 'rob_score', 'fin_sentiment'],
      dtype='object')

In [4]:
WFC = roberta_data[roberta_data['Ticker']=='WFC']
WFC= WFC[WFC['Publishing Time'].notna()]
WFC[['Headline','Text', 'rob_sentiment', 'rob_score']].iloc[0]['Text']

"We learned this week that the scandal-plagued bank's CEO received about 5.7% more in compensation in 2018."

import intraday data. Note that the intraday times are extended stock hours, not just the core hours. Also, data starts from March 15, 2019 to March 15, 2024 (different range than the previous daily stock data)

intraday data is split by stock. For now, import AAPL but want to do it for all eventually.

In [5]:
AAPL_data = pd.read_csv('./Intraday_StockData/AAPL_intraday.csv')
AAPL_data.head()

,timestamp,open,high,low,close,volume,numtrades,vwap
0,2019-03-15T08:00:00Z,184.25,184.50,184.25,184.50,2381,37,184.253452
1,2019-03-15T08:15:00Z,184.50,184.50,184.45,184.50,1020,6,184.467647
2,2019-03-15T08:30:00Z,184.63,184.68,184.63,184.68,761,8,184.666058
3,2019-03-15T08:45:00Z,184.69,185.33,184.69,184.70,16025,87,184.798192
4,2019-03-15T09:00:00Z,184.69,184.69,184.50,184.58,2076,10,184.589908


In [6]:
AAPL_data['timestamp'] = pd.to_datetime(AAPL_data['timestamp'])
AAPL_data.head()

,timestamp,open,high,low,close,volume,numtrades,vwap
0,2019-03-15 08:00:00+00:00,184.25,184.50,184.25,184.50,2381,37,184.253452
1,2019-03-15 08:15:00+00:00,184.50,184.50,184.45,184.50,1020,6,184.467647
2,2019-03-15 08:30:00+00:00,184.63,184.68,184.63,184.68,761,8,184.666058
3,2019-03-15 08:45:00+00:00,184.69,185.33,184.69,184.70,16025,87,184.798192
4,2019-03-15 09:00:00+00:00,184.69,184.69,184.50,184.58,2076,10,184.589908


In [7]:
AAPL_data.set_index('timestamp', inplace = True)

In [8]:
AAPL_data.loc['2020-12-24']

,open,high,low,close,volume,numtrades,vwap
timestamp,,,,,,,
2020-12-24 00:00:00+00:00,130.9000,131.0300,130.9000,130.9899,22568,201,130.982653
2020-12-24 00:15:00+00:00,130.9800,130.9900,130.9100,130.9100,17337,198,130.950504
2020-12-24 00:30:00+00:00,130.9100,130.9600,130.8900,130.9500,17816,259,130.924962
2020-12-24 00:45:00+00:00,130.9500,131.0500,130.9200,131.0500,37346,321,130.981591
2020-12-24 09:00:00+00:00,131.0500,131.1500,131.0000,131.0500,8330,116,131.044055
2020-12-24 09:15:00+00:00,131.0500,131.1000,131.0000,131.0000,3628,58,131.041621
2020-12-24 09:30:00+00:00,131.0000,131.0900,131.0000,131.0900,4678,35,131.022422
2020-12-24 09:45:00+00:00,131.0000,131.0000,131.0000,131.0000,293,5,131.000273
2020-12-24 10:00:00+00:00,131.0100,131.0200,131.0000,131.0000,1478,26,131.008843


restrict to only article data with data from relevant stock

In [9]:
AAPL_roberta = roberta_data[(roberta_data['Ticker']=='AAPL') & (roberta_data['Publishing Time'].notna())]
AAPL_roberta.head()

,Market Date,Publishing Time,Ticker,Sector,finvader_neg,finvader_neu,finvader_pos,finvader_tot,Source,Headline,...,Open,High,Low,Close,Volume,Dividends,Stock Splits,rob_sentiment,rob_score,fin_sentiment
162,2019-03-15,2019-03-15 10:47:26+00:00,AAPL,Technology,0.000,0.660,0.340,0.0396,The Motley Fool,Don't Underestimate Apple's iPhone Business,...,44.428467,45.024531,44.161680,44.733707,156171600,0.0,0.0,positive,0.669296,neutral
183,2019-03-18,2019-03-15 19:16:44+00:00,AAPL,Technology,0.000,0.664,0.336,0.8481,CNBC,Apple's new streaming service seeks Oscar glory,...,44.656790,45.279293,44.654385,45.190365,104879200,0.0,0.0,neutral,0.999767,positive
184,2019-03-18,2019-03-15 19:40:00+00:00,AAPL,Technology,0.054,0.946,0.000,-0.0086,The Motley Fool,Apple Just Scored the $1 Billion It Was Asking...,...,44.656790,45.279293,44.654385,45.190365,104879200,0.0,0.0,neutral,0.999792,neutral
190,2019-03-19,2019-03-18 20:39:58+00:00,AAPL,Technology,0.000,0.896,0.104,0.0361,CNBC,Apple's heart study signals a new era of medic...,...,45.269685,45.423508,44.685636,44.832249,126585600,0.0,0.0,neutral,0.999752,neutral
194,2019-03-19,2019-03-19 00:04:51+00:00,AAPL,Technology,0.026,0.685,0.289,0.6575,The Motley Fool,2 Big iPad Updates Revealed: Here's the Skinny...,...,45.269685,45.423508,44.685636,44.832249,126585600,0.0,0.0,positive,0.999705,positive


In [10]:
AAPL_roberta.columns

Index(['Market Date', 'Publishing Time', 'Ticker', 'Sector', 'finvader_neg',
       'finvader_neu', 'finvader_pos', 'finvader_tot', 'Source', 'Headline',
       'Text', 'URL', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'rob_sentiment', 'rob_score', 'fin_sentiment'],
      dtype='object')

In [11]:
relevant_cols = ['Publishing Time', 'Ticker', 'Sector', 'Source', 'Headline',
       'Text', 'URL', 'rob_sentiment', 'rob_score', 'fin_sentiment','finvader_neg',
       'finvader_neu', 'finvader_pos', 'finvader_tot']


In [12]:
AAPL_roberta = AAPL_roberta[relevant_cols]
AAPL_roberta.head()

,Publishing Time,Ticker,Sector,Source,Headline,Text,URL,rob_sentiment,rob_score,fin_sentiment,finvader_neg,finvader_neu,finvader_pos,finvader_tot
162,2019-03-15 10:47:26+00:00,AAPL,Technology,The Motley Fool,Don't Underestimate Apple's iPhone Business,The segment is an invaluable asset to Apple's ...,https://www.fool.com/investing/2019/03/14/dont...,positive,0.669296,neutral,0.000,0.660,0.340,0.0396
183,2019-03-15 19:16:44+00:00,AAPL,Technology,CNBC,Apple's new streaming service seeks Oscar glory,Apple is reportedly hiring strategists to help...,https://www.cnbc.com/2019/03/15/apples-new-str...,neutral,0.999767,positive,0.000,0.664,0.336,0.8481
184,2019-03-15 19:40:00+00:00,AAPL,Technology,The Motley Fool,Apple Just Scored the $1 Billion It Was Asking...,But it won't be able to collect quite yet.,https://www.fool.com/investing/2019/03/15/appl...,neutral,0.999792,neutral,0.054,0.946,0.000,-0.0086
190,2019-03-18 20:39:58+00:00,AAPL,Technology,CNBC,Apple's heart study signals a new era of medic...,Stanford's researchers talked about the study ...,https://www.cnbc.com/2019/03/18/apple-watch-he...,neutral,0.999752,neutral,0.000,0.896,0.104,0.0361
194,2019-03-19 00:04:51+00:00,AAPL,Technology,The Motley Fool,2 Big iPad Updates Revealed: Here's the Skinny...,iPad sales grew faster during the holiday quar...,https://www.fool.com/investing/2019/03/18/appl...,positive,0.999705,positive,0.026,0.685,0.289,0.6575


create filter to extract only core stock market hours

In [13]:
master_stock = pd.read_csv('./data/master_stock.csv')
freq = datetime.timedelta(minutes = 15)
core_times = master_stock[['Opening Time', 'Closing Time']].apply(lambda row: pd.date_range(row['Opening Time'], row['Closing Time'], freq =freq ), axis = 1)
core_times = core_times.explode()

In [14]:
AAPL_data = AAPL_data.filter(items = core_times, axis = 0)

In [15]:
AAPL_data.head()

,open,high,low,close,volume,numtrades,vwap
2019-03-15 13:30:00+00:00,184.78,184.92,183.740,184.2534,13671352,15879,184.625306
2019-03-15 13:45:00+00:00,184.25,184.99,184.035,184.9100,1494556,10938,184.590954
2019-03-15 14:00:00+00:00,184.91,185.01,184.430,184.8600,1431883,10369,184.780349
2019-03-15 14:15:00+00:00,184.86,185.34,184.690,185.2500,1151525,8257,185.085017
2019-03-15 14:30:00+00:00,185.25,185.28,184.910,185.2601,927367,7238,185.094187


In [26]:
AAPL_data.reset_index(inplace = True)
AAPL_data.tail()

,level_0,index,open,high,low,close,volume,numtrades,vwap
34013,34013,2024-03-15 19:00:00+00:00,171.5256,171.78,171.470,171.6250,1619750,17649,171.554619
34014,34014,2024-03-15 19:15:00+00:00,171.6250,171.99,171.470,171.8650,1550044,17146,171.698291
34015,34015,2024-03-15 19:30:00+00:00,171.8560,172.00,171.755,171.7799,1913286,21338,171.871088
34016,34016,2024-03-15 19:45:00+00:00,171.7700,172.60,171.478,172.5600,6766652,57205,172.179735
34017,34017,2024-03-15 20:00:00+00:00,172.5700,172.62,172.400,172.6100,47806712,2115,172.619752


In [23]:
AAPL_roberta['Publishing Time']= pd.to_datetime(AAPL_roberta['Publishing Time'])
AAPL_roberta= AAPL_roberta.sort_values('Publishing Time')
AAPL_roberta.head()

,Publishing Time,Ticker,Sector,Source,Headline,Text,URL,rob_sentiment,rob_score,fin_sentiment,finvader_neg,finvader_neu,finvader_pos,finvader_tot
162,2019-03-15 10:47:26+00:00,AAPL,Technology,The Motley Fool,Don't Underestimate Apple's iPhone Business,The segment is an invaluable asset to Apple's ...,https://www.fool.com/investing/2019/03/14/dont...,positive,0.669296,neutral,0.000,0.660,0.340,0.0396
183,2019-03-15 19:16:44+00:00,AAPL,Technology,CNBC,Apple's new streaming service seeks Oscar glory,Apple is reportedly hiring strategists to help...,https://www.cnbc.com/2019/03/15/apples-new-str...,neutral,0.999767,positive,0.000,0.664,0.336,0.8481
184,2019-03-15 19:40:00+00:00,AAPL,Technology,The Motley Fool,Apple Just Scored the $1 Billion It Was Asking...,But it won't be able to collect quite yet.,https://www.fool.com/investing/2019/03/15/appl...,neutral,0.999792,neutral,0.054,0.946,0.000,-0.0086
218,2019-03-18 14:54:26+00:00,AAPL,Technology,CNBC,Apple unveils new iPad Air and iPad mini ahead...,The upgraded models offer keyboard support for...,https://www.cnbc.com/2019/03/18/apple-unveils-...,neutral,0.999518,neutral,0.000,0.858,0.142,0.0348
212,2019-03-18 15:29:38+00:00,AAPL,Technology,Seeking Alpha,Buy Apple For Great Total Return And Future 10...,This article is about Apple and why it's a buy...,https://seekingalpha.com/article/4249322-buy-a...,positive,0.986010,positive,0.000,0.692,0.308,0.4176


In [33]:
df_for = pd.merge_asof(AAPL_roberta, AAPL_data, right_on='index', left_on='Publishing Time', direction = 'forward')
df_for.head()

,Publishing Time,Ticker,Sector,Source,Headline,Text,URL,rob_sentiment,rob_score,fin_sentiment,...,finvader_tot,level_0,index,open,high,low,close,volume,numtrades,vwap
0,2019-03-15 10:47:26+00:00,AAPL,Technology,The Motley Fool,Don't Underestimate Apple's iPhone Business,The segment is an invaluable asset to Apple's ...,https://www.fool.com/investing/2019/03/14/dont...,positive,0.669296,neutral,...,0.0396,0.0,2019-03-15 13:30:00+00:00,184.78,184.92,183.74,184.2534,13671352.0,15879.0,184.625306
1,2019-03-15 19:16:44+00:00,AAPL,Technology,CNBC,Apple's new streaming service seeks Oscar glory,Apple is reportedly hiring strategists to help...,https://www.cnbc.com/2019/03/15/apples-new-str...,neutral,0.999767,positive,...,0.8481,24.0,2019-03-15 19:30:00+00:00,186.99,187.09,186.70,186.8300,1195278.0,8685.0,186.869055
2,2019-03-15 19:40:00+00:00,AAPL,Technology,The Motley Fool,Apple Just Scored the $1 Billion It Was Asking...,But it won't be able to collect quite yet.,https://www.fool.com/investing/2019/03/15/appl...,neutral,0.999792,neutral,...,-0.0086,25.0,2019-03-15 19:45:00+00:00,186.83,186.84,185.89,186.2200,3430174.0,24024.0,186.197377
3,2019-03-18 14:54:26+00:00,AAPL,Technology,CNBC,Apple unveils new iPad Air and iPad mini ahead...,The upgraded models offer keyboard support for...,https://www.cnbc.com/2019/03/18/apple-unveils-...,neutral,0.999518,neutral,...,0.0348,33.0,2019-03-18 15:00:00+00:00,188.22,188.23,187.75,187.8600,791500.0,6012.0,187.961633
4,2019-03-18 15:29:38+00:00,AAPL,Technology,Seeking Alpha,Buy Apple For Great Total Return And Future 10...,This article is about Apple and why it's a buy...,https://seekingalpha.com/article/4249322-buy-a...,positive,0.986010,positive,...,0.4176,35.0,2019-03-18 15:30:00+00:00,187.62,187.62,187.19,187.2400,850945.0,5820.0,187.335396


In [34]:
diff = df_for['Publishing Time'].dt.day != df_for['index'].dt.day
df_for[diff]

,Publishing Time,Ticker,Sector,Source,Headline,Text,URL,rob_sentiment,rob_score,fin_sentiment,...,finvader_tot,level_0,index,open,high,low,close,volume,numtrades,vwap
10,2019-03-18 20:39:58+00:00,AAPL,Technology,CNBC,Apple's heart study signals a new era of medic...,Stanford's researchers talked about the study ...,https://www.cnbc.com/2019/03/18/apple-watch-he...,neutral,0.999752,neutral,...,0.0361,54.0,2019-03-19 13:30:00+00:00,188.24,188.99,188.00,188.5200,3133369.0,17735.0,188.515239
19,2019-03-19 22:01:30+00:00,AAPL,Technology,The Motley Fool,Revealed: The Key Metric Showing Apple Has Los...,Is it better to be an innovator or a tastemake...,https://www.fool.com/investing/2019/03/19/reve...,negative,0.967033,neutral,...,0.0900,81.0,2019-03-20 13:30:00+00:00,186.28,186.58,185.18,186.0600,2290118.0,13704.0,185.940406
20,2019-03-19 22:18:16+00:00,AAPL,Technology,The Motley Fool,Apple Has Another Huawei Problem,Apple's iPhone unit shipment woes could cause ...,https://www.fool.com/investing/2019/03/19/appl...,negative,0.998527,negative,...,-0.4570,81.0,2019-03-20 13:30:00+00:00,186.28,186.58,185.18,186.0600,2290118.0,13704.0,185.940406
40,2019-03-21 20:17:00+00:00,AAPL,Technology,CNBC,Apple's stock is on a tear as it builds suspen...,Apple also came close to passing Microsoft aga...,https://www.cnbc.com/2019/03/21/apple-rallies-...,positive,0.755562,neutral,...,0.0142,135.0,2019-03-22 13:30:00+00:00,195.34,197.69,194.79,196.8100,5279917.0,31279.0,196.481756
47,2019-03-22 20:04:42+00:00,AAPL,Technology,Investors Business Daily,Apple Might Differentiate Its Streaming Video ...,Apple will raise the curtain on its long-await...,https://feedproxy.google.com/~r/BusinessRss/~3...,positive,0.999085,positive,...,0.3899,162.0,2019-03-25 13:30:00+00:00,191.64,191.98,188.69,188.9358,4766477.0,31145.0,190.698052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12977,2024-03-14 20:18:13+00:00,AAPL,Technology,Seeking Alpha,Apple Reenters Our Buy Zone (Technical Analysi...,Apple Inc. stock prices have corrected to arou...,https://seekingalpha.com/article/4678303-apple...,positive,0.996985,neutral,...,0.0271,33991.0,2024-03-15 13:30:00+00:00,171.17,172.48,171.00,171.4200,18669209.0,69694.0,171.186447
12978,2024-03-14 22:51:06+00:00,AAPL,Technology,Zacks Investment Research,Apple (AAPL) Advances While Market Declines: S...,Apple (AAPL) closed at $173 in the latest trad...,https://www.zacks.com/stock/news/2241013/apple...,positive,0.991740,negative,...,-0.3538,33991.0,2024-03-15 13:30:00+00:00,171.17,172.48,171.00,171.4200,18669209.0,69694.0,171.186447
12988,2024-03-15 21:03:01+00:00,AAPL,Technology,Business Insider,Tim Cook's comments about iPhone sales in Chin...,Apple has agreed to pay $490 million to settle...,https://www.businessinsider.com/tim-cooks-comm...,positive,0.996090,positive,...,0.2789,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12989,2024-03-15 22:18:35+00:00,AAPL,Technology,The Guardian,Apple to pay $490m to settle claims it misled ...,Company denied that Tim Cook deceived investor...,https://www.theguardian.com/technology/2024/ma...,negative,0.773549,negative,...,-0.6772,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Using full intraday data

Just testing code again

In [78]:
full_AAPL_data = pd.read_csv('./Intraday_StockData/AAPL_intraday.csv')
full_AAPL_data.head()

,timestamp,open,high,low,close,volume,numtrades,vwap
0,2019-03-15T08:00:00Z,184.25,184.50,184.25,184.50,2381,37,184.253452
1,2019-03-15T08:15:00Z,184.50,184.50,184.45,184.50,1020,6,184.467647
2,2019-03-15T08:30:00Z,184.63,184.68,184.63,184.68,761,8,184.666058
3,2019-03-15T08:45:00Z,184.69,185.33,184.69,184.70,16025,87,184.798192
4,2019-03-15T09:00:00Z,184.69,184.69,184.50,184.58,2076,10,184.589908


In [36]:
full_AAPL_data['timestamp'] = pd.to_datetime(full_AAPL_data['timestamp'])

In [43]:
roberta_data = pd.read_csv('./data/prebuilt_rob_sentiment.csv')
AAPL_roberta = roberta_data[(roberta_data['Ticker']=='AAPL') & (roberta_data['Publishing Time'].notna())]
AAPL_roberta['Publishing Time'] = pd.to_datetime(AAPL_roberta['Publishing Time'])
AAPL_roberta = AAPL_roberta.sort_values('Publishing Time')

/var/folders/tg/nkrt27q937bgzsnrzszxn7bc0000gn/T/ipykernel_4792/4028695255.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AAPL_roberta['Publishing Time'] = pd.to_datetime(AAPL_roberta['Publishing Time'])


In [61]:
df = pd.merge_asof(AAPL_roberta,full_AAPL_data, right_on='timestamp', left_on='Publishing Time', direction = 'forward')
df.head()

,Market Date,Publishing Time,Ticker,Sector,finvader_neg,finvader_neu,finvader_pos,finvader_tot,Source,Headline,...,rob_score,fin_sentiment,timestamp,open,high,low,close,volume,numtrades,vwap
0,2019-03-15,2019-03-15 10:47:26+00:00,AAPL,Technology,0.000,0.660,0.340,0.0396,The Motley Fool,Don't Underestimate Apple's iPhone Business,...,0.669296,neutral,2019-03-15 11:00:00+00:00,185.12,185.12,184.72,184.90,26592.0,156.0,184.853703
1,2019-03-18,2019-03-15 19:16:44+00:00,AAPL,Technology,0.000,0.664,0.336,0.8481,CNBC,Apple's new streaming service seeks Oscar glory,...,0.999767,positive,2019-03-15 19:30:00+00:00,186.99,187.09,186.70,186.83,1195278.0,8685.0,186.869055
2,2019-03-18,2019-03-15 19:40:00+00:00,AAPL,Technology,0.054,0.946,0.000,-0.0086,The Motley Fool,Apple Just Scored the $1 Billion It Was Asking...,...,0.999792,neutral,2019-03-15 19:45:00+00:00,186.83,186.84,185.89,186.22,3430174.0,24024.0,186.197377
3,2019-03-19,2019-03-18 14:54:26+00:00,AAPL,Technology,0.000,0.858,0.142,0.0348,CNBC,Apple unveils new iPad Air and iPad mini ahead...,...,0.999518,neutral,2019-03-18 15:00:00+00:00,188.22,188.23,187.75,187.86,791500.0,6012.0,187.961633
4,2019-03-19,2019-03-18 15:29:38+00:00,AAPL,Technology,0.000,0.692,0.308,0.4176,Seeking Alpha,Buy Apple For Great Total Return And Future 10...,...,0.986010,positive,2019-03-18 15:30:00+00:00,187.62,187.62,187.19,187.24,850945.0,5820.0,187.335396


In [46]:
diff = df['Publishing Time'].dt.day != df['timestamp'].dt.day
df[diff]

,Market Date,Publishing Time,Ticker,Sector,finvader_neg,finvader_neu,finvader_pos,finvader_tot,Source,Headline,...,rob_score,fin_sentiment,timestamp,open,high,low,close,volume,numtrades,vwap
50,2019-03-25,2019-03-23 10:21:00+00:00,AAPL,Technology,0.064,0.902,0.033,-0.0013,CNBC,"In China, Apple's Cook says he's bullish on gl...",...,0.999554,neutral,2019-03-25 08:00:00+00:00,190.37,190.37,190.00,190.01,3301.0,50.0,190.089470
51,2019-03-25,2019-03-23 15:00:00+00:00,AAPL,Technology,0.000,0.821,0.179,0.0347,The Motley Fool,Apple's Video-Streaming Service Might Just Be ...,...,0.999505,neutral,2019-03-25 08:00:00+00:00,190.37,190.37,190.00,190.01,3301.0,50.0,190.089470
52,2019-03-25,2019-03-24 11:37:00+00:00,AAPL,Technology,0.000,0.821,0.179,0.3798,The Motley Fool,Why Apple Won't Be the Next Netflix,...,0.998638,positive,2019-03-25 08:00:00+00:00,190.37,190.37,190.00,190.01,3301.0,50.0,190.089470
53,2019-03-25,2019-03-24 15:31:00+00:00,AAPL,Technology,0.000,0.882,0.118,0.0400,CNBC,Steve Jobs was trying to revolutionize TV almo...,...,0.999831,neutral,2019-03-25 08:00:00+00:00,190.37,190.37,190.00,190.01,3301.0,50.0,190.089470
54,2019-03-25,2019-03-24 21:18:37+00:00,AAPL,Technology,0.000,0.821,0.179,0.0464,Market Watch,"Apple set to unveil streaming service, will re...",...,0.999827,neutral,2019-03-25 08:00:00+00:00,190.37,190.37,190.00,190.01,3301.0,50.0,190.089470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12940,2024-03-11,2024-03-10 07:30:00+00:00,AAPL,Technology,0.098,0.734,0.168,0.0167,The Motley Fool,2 Reasons to Buy Apple Stock Like There's No T...,...,0.998487,neutral,2024-03-11 08:00:00+00:00,171.00,171.66,171.00,171.12,12008.0,624.0,171.368997
12941,2024-03-11,2024-03-10 12:35:00+00:00,AAPL,Technology,0.097,0.699,0.205,0.0310,The Motley Fool,Apple Could Launch Its Most Important Product ...,...,0.970894,neutral,2024-03-11 08:00:00+00:00,171.00,171.66,171.00,171.12,12008.0,624.0,171.368997
12978,2024-03-15,2024-03-14 22:51:06+00:00,AAPL,Technology,0.088,0.841,0.071,-0.3538,Zacks Investment Research,Apple (AAPL) Advances While Market Declines: S...,...,0.991740,negative,2024-03-15 08:00:00+00:00,173.41,173.52,173.21,173.50,9689.0,449.0,173.322421
12989,2024-03-18,2024-03-15 22:18:35+00:00,AAPL,Technology,0.211,0.618,0.171,-0.6772,The Guardian,Apple to pay $490m to settle claims it misled ...,...,0.773549,negative,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
merge_col = list(full_AAPL_data.columns)
full_AAPL_data['timestamp'] = pd.to_datetime(full_AAPL_data['timestamp'])
df_a = pd.merge(df, full_AAPL_data, on = merge_col, how = 'outer')
df_a.head()

# len(df_a['timestamp'].unique()), len(full_AAPL_data['timestamp'])
#df_a first two articles are same event, two different sources, two different sentiment. maybe worth looking into


,Market Date,Publishing Time,Ticker,Sector,finvader_neg,finvader_neu,finvader_pos,finvader_tot,Source,Headline,...,rob_score,fin_sentiment,timestamp,open,high,low,close,volume,numtrades,vwap
0,2024-03-18,2024-03-15 22:18:35+00:00,AAPL,Technology,0.211,0.618,0.171,-0.6772,The Guardian,Apple to pay $490m to settle claims it misled ...,...,0.773549,negative,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-03-18,2024-03-15 23:21:00+00:00,AAPL,Technology,0.084,0.784,0.133,-0.0798,Market Watch,Apple to pay $490 million to settle claims it ...,...,0.984778,neutral,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2019-03-15 08:00:00+00:00,184.25,184.50,184.25,184.50,2381.0,37.0,184.253452
3,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2019-03-15 08:15:00+00:00,184.50,184.50,184.45,184.50,1020.0,6.0,184.467647
4,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2019-03-15 08:30:00+00:00,184.63,184.68,184.63,184.68,761.0,8.0,184.666058


articles in beginning of df_a are those which are associated to stock days after our intraday stock data. Moreover, the two articles in this case are of the same event, but two different sentiments based on wording of the article. May be of future interest.

Now lets generate all the relevant csv files

In [48]:
tickers = roberta_data['Ticker'].unique()
tickers

array(['JNJ', 'JPM', 'WFC', 'V', 'UNH', 'NVDA', 'MSFT', 'MA', 'MRK',
       'ABBV', 'AMZN', 'BAC', 'LLY', 'GOOGL', 'AAPL'], dtype=object)

In [50]:
relevant_cols = ['Publishing Time', 'Ticker', 'Sector', 'Source', 'Headline',
       'Text', 'URL', 'fin_sentiment', 'finvader_tot', 'rob_sentiment', 'rob_score',]
roberta_data = roberta_data[relevant_cols]
roberta_data['Publishing Time'] = pd.to_datetime(roberta_data['Publishing Time'])
roberta_data.head()

,Publishing Time,Ticker,Sector,Source,Headline,Text,URL,fin_sentiment,finvader_tot,rob_sentiment,rob_score
0,NaT,JNJ,Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,JPM,Finance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaT,WFC,Finance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,V,Finance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaT,UNH,Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
for tick in tickers:
    intraday_data = pd.read_csv('./Intraday_StockData/'+tick+'_intraday.csv')
    intraday_data['timestamp'] = pd.to_datetime(intraday_data['timestamp'])
    tick_rob = roberta_data[(roberta_data['Ticker']==tick) & (roberta_data['Publishing Time'].notna())]
    tick_rob = tick_rob.sort_values('Publishing Time')
    df1 = pd.merge_asof(tick_rob, intraday_data, left_on = 'Publishing Time', right_on = 'timestamp', direction = 'forward')
    merge_col = list(intraday_data.columns)
    complete_df = pd.merge(df1, intraday_data, on = merge_col, how = 'outer')
    complete_df.rename(columns = {'timestamp':'stock_time'}, inplace = True)
    complete_df.to_csv('./prebuilt_rob_data/'+tick+'_prerob.csv', index = False)


    


In [89]:
df = pd.read_csv('./prebuilt_rob_data/AAPL_prerob.csv')
df.head()

,Publishing Time,Ticker,Sector,Source,Headline,Text,URL,fin_sentiment,finvader_tot,rob_sentiment,rob_score,stock_time,open,high,low,close,volume,numtrades,vwap
0,2024-03-15 22:18:35+00:00,AAPL,Technology,The Guardian,Apple to pay $490m to settle claims it misled ...,Company denied that Tim Cook deceived investor...,https://www.theguardian.com/technology/2024/ma...,negative,-0.6772,negative,0.773549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-03-15 23:21:00+00:00,AAPL,Technology,Market Watch,Apple to pay $490 million to settle claims it ...,Apple Inc. has agreed to pay $490 million to s...,https://www.marketwatch.com/story/apple-to-pay...,neutral,-0.0798,positive,0.984778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-15 08:00:00+00:00,184.25,184.50,184.25,184.50,2381.0,37.0,184.253452
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-15 08:15:00+00:00,184.50,184.50,184.45,184.50,1020.0,6.0,184.467647
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-15 08:30:00+00:00,184.63,184.68,184.63,184.68,761.0,8.0,184.666058
